In [ ]:
import torch
from typing import Iterable, Callable, Optional, List
from torch.nn import grad as nn_grad


@torch.no_grad()
def batched_cg_conv(X_act, Y2_scaled, G, module: torch.nn.Conv2d, num_iters: int, eps=1e-6, denom_eps=1e-10):
    """
    Solve (X_act^T @ Y2_scaled @ X_act + eps * I)W = G for every output channel (row-wise) using CG.
    """

    W = torch.zeros_like(G)
    R = G.clone()
    P = R.clone()

    rsold = (R * R).sum(dim=(1, 2, 3), keepdim=True)

    for _ in range(num_iters):
        XP = torch.nn.functional.conv2d(
            X_act, P,
            stride=module.stride,
            padding=module.padding,
            dilation=module.dilation,
            groups=module.groups,
        )
        XP.mul_(Y2_scaled)
        Ap = nn_grad.conv2d_weight(
            X_act, P.shape, XP,
            stride=module.stride,
            padding=module.padding,
            dilation=module.dilation,
            groups=module.groups,
        )
        Ap.add_(P, alpha=eps)

        pAp = (P * Ap).sum(dim=(1, 2, 3), keepdim=True)
        alpha = rsold / pAp.add_(denom_eps)

        W.addcmul_(P, alpha)
        R.addcmul_(Ap, alpha, value=-1.0)

        rsnew = (R * R).sum(dim=(1, 2, 3), keepdim=True)
        beta = rsnew / rsold.add_(denom_eps)

        P.mul_(beta).add_(R)
        rsold = rsnew

    return W


@torch.no_grad()
def fisher_normalized_trace(X, Y2dB, module, ones):
    """
    Compute normalized trace of row-wise Fisher matrices.
    """
    assert module.groups == 1, "FROG currently only supports groups=1"

    c_out, c_in, kh, kw = module.weight.shape
    x2_sum = torch.linalg.vecdot(X, X, dim=1)
    window_sum = torch.nn.functional.conv2d(
        x2_sum.unsqueeze(1),
        ones,
        stride=module.stride,
        padding=module.padding,
        dilation=module.dilation
    )
    scalar = (Y2dB * window_sum).sum(dim=(0, 2, 3))
    normalization = c_in * kh * kw
    return scalar.div_(normalization)


class Frog(torch.optim.Optimizer):
    def __init__(
            self,
            params: Iterable[torch.Tensor],
            model: torch.nn.Module,
            lr: float = 1e-3,
            lr_other_ratio: float = 0.25,
            momentum: float = 0.0,
            nesterov: bool = False,
            weight_decay: float = 0.0,
            sample_size: Optional[int] = 24,
            num_iters: int = 4,
            tau: float = 1e-6,
            tau_final: Optional[float] = None,
            tau_steps: int = 15 * 50,
            batch_averaged: bool = True,
            skip_modules: List[str] = None,
            denom_eps: float = 1e-10
    ):
        """
        FROG: Fisher ROw-wise PreconditioninG.

        Current implementation:
          - Conv2d only (Linear not supported yet)
          - Iterative solve for row-wise empirical Fisher matrices using batched CG.
          - Random subsampling of activations (X and Y)

        :param params: Optimized parameters
        :param model: PyTorch model (required for hooks)
        :param lr: Learning rate for preconditioned modules
        :param lr_other_ratio: Ratio for learning rate of other modules (lr_other = lr * lr_other_ratio)
        :param momentum: Momentum (same as in SGD)
        :param nesterov: Nesterov momentum (same as in SGD)
        :param weight_decay: Weight decay (same as in SGD)
        :param sample_size: Number of activations to subsample for iterative CG. None means keep all activations (not recommended).
        :param num_iters: Number of CG iterations
        :param tau: Fisher damping factor: damping = tr(F)/D * tau
        :param batch_averaged: Whether loss has reduction="mean" (for rescaling)
        :param skip_modules: Names of modules to skip
        :param denom_eps: denominator epsilon for numerical stability
        """
        if lr < 0.0:
            raise ValueError(f"Invalid learning rate: {lr}")
        if lr_other_ratio < 0.0:
            raise ValueError(f"Invalid lr_other_ratio: {lr_other_ratio}")
        if momentum < 0.0:
            raise ValueError(f"Invalid momentum value: {momentum}")
        if nesterov and momentum <= 0.0:
            raise ValueError("Nesterov requires momentum > 0.")
        if sample_size is not None and sample_size <= 0:
            raise ValueError(f"sample_size must be None or positive, got {sample_size}")
        if num_iters <= 0:
            raise ValueError(f"num_iters must be positive, got {num_iters}")
        if tau < 0:
            raise ValueError(f"Invalid tau value: {tau}")
        if denom_eps < 0:
            raise ValueError(f"Invalid denominator epsilon value: {denom_eps}")

        self.tau_final = tau_final
        self.tau_steps = tau_steps

        defaults = dict(
            lr=lr,
            lr_other_ratio=lr_other_ratio,
            momentum=momentum,
            nesterov=nesterov,
            weight_decay=weight_decay,
            sample_size=sample_size,
            num_iters=num_iters,
            tau=tau,
            batch_averaged=batch_averaged,
            denom_eps=denom_eps
        )
        super().__init__(params, defaults)

        self.sample_indices: dict[torch.nn.Module, torch.Tensor] = {}
        self.out_grads: dict[torch.nn.Module, torch.Tensor] = {}
        self.in_acts: dict[torch.nn.Module, torch.Tensor] = {}
        self.param_to_module: dict[torch.Tensor, torch.nn.Module] = {}
        self.hooks: list[torch.utils.hooks.RemovableHandle] = []
        self._ones_buffer = {}

        skip_modules = skip_modules or []

        for name, module in model.named_modules():
            if name not in skip_modules:
                if isinstance(module, torch.nn.Conv2d):
                    h_fwd = module.register_forward_hook(self._save_input)
                    h_bwd = module.register_full_backward_hook(self._save_grad_outputs)
                    self.hooks.extend([h_fwd, h_bwd])
                    self.param_to_module[module.weight] = module

    def _get_ones(self, kh, kw, device, dtype):
        key = (kh, kw, device, dtype)
        if key not in self._ones_buffer:
            self._ones_buffer[key] = torch.ones((1, 1, kh, kw), device=device, dtype=dtype)
        return self._ones_buffer[key]

    def _save_input(self, module: torch.nn.Module, input, output):
        if not torch.is_grad_enabled():
            return
        if len(input) == 0:
            return

        with torch.no_grad():
            X_all = input[0]

            if isinstance(module, torch.nn.Conv2d):
                B_all = X_all.shape[0]
                sample_size = self.defaults['sample_size']
                if sample_size is not None and sample_size < B_all:
                    idx = torch.randperm(X_all.shape[0], device=X_all.device)[:sample_size]
                    self.sample_indices[module] = idx
                    self.in_acts[module] = X_all[idx].detach()
                else:
                    self.in_acts[module] = X_all.detach().clone()

    def _save_grad_outputs(self, module: torch.nn.Module, grad_input, grad_output):
        if not grad_output or grad_output[0] is None:
            return

        with torch.no_grad():
            Y_all = grad_output[0]

            if isinstance(module, torch.nn.Conv2d):
                if module in self.sample_indices:
                    idx = self.sample_indices.pop(module)
                    Y = Y_all[idx].detach()
                else:
                    Y = Y_all.detach().clone()

                B_all = Y_all.shape[0]
                B = Y.shape[0]

                if self.defaults['batch_averaged']:
                    scale = (B_all * B_all) / B
                    Y2dB = Y.square_().mul_(scale)
                else:
                    Y2dB = Y.square_().div_(B)

                self.out_grads[module] = Y2dB


    @torch.no_grad()
    def step(self, closure=None):
        loss = None
        if closure is not None:
            with torch.enable_grad():
                loss = closure()

        for group in self.param_groups:
            lr = group["lr"]
            lr_other_ratio = group['lr_other_ratio']
            momentum = group["momentum"]
            nesterov = group['nesterov']
            weight_decay = group['weight_decay']
            sample_size = group['sample_size']
            num_iters = group['num_iters']
            tau_initial = group['tau']
            batch_averaged = group['batch_averaged']
            denom_eps = group['denom_eps']

            for p in group["params"]:
                if p.grad is None:
                    continue
                if p.grad.is_sparse:
                    raise RuntimeError("Frog does not support sparse gradients")

                grad = p.grad

                state = self.state[p]
                if len(state) == 0:
                    state["step"] = 0
                    if momentum > 0.0:
                        state["momentum_buffer"] = torch.zeros_like(p)

                state["step"] += 1
                if self.tau_final is not None:
                    frac = state['step'] / self.tau_steps
                    if frac > 1:
                        tau = self.tau_final
                    else:
                        tau = tau_initial * (self.tau_final / tau_initial) ** frac
                    # if random.random() > 0.999:
                    #     print(f'tau: {tau:.6f}')
                else:
                    tau = tau_initial

                if momentum > 0.0:
                    buf = state["momentum_buffer"]
                    buf.mul_(momentum).add_(grad)
                    eff_grad = grad.add(buf, alpha=momentum) if nesterov else buf
                else:
                    eff_grad = grad

                if p in self.param_to_module:
                    module = self.param_to_module[p]

                    if isinstance(module, torch.nn.Linear):
                        raise Exception("Linear not supported yet")
                    elif isinstance(module, torch.nn.Conv2d):
                        X = self.in_acts[module]
                        Y2dB = self.out_grads[module]

                        F_tr = fisher_normalized_trace(X, Y2dB, module, ones=self._get_ones(p.shape[-2], p.shape[-1], X.device, X.dtype))
                        Y2_scaled = Y2dB.div_(F_tr.add_(denom_eps).view(1, -1, 1, 1))

                        # Solve row-wise (F/F_tr + tau * I)W = G using CG
                        P1 = batched_cg_conv(
                            X, Y2_scaled, eff_grad,
                            module=module,
                            num_iters=num_iters,
                            eps=tau,
                            denom_eps=denom_eps
                        )

                        # Normalization
                        P1_dot = (P1 * eff_grad).sum(dim=(1, 2, 3))
                        scalar = P1_dot.add_(denom_eps).rsqrt_()

                        if weight_decay > 0.0:
                            p.mul_(1 - lr * weight_decay)
                        p.addcmul_(P1, scalar.view(-1, 1, 1, 1), value=-lr)
                    else:
                        raise ValueError(f"Invalid module: {module}")
                else:
                    direction = eff_grad
                    if weight_decay > 0.0:
                        direction.add_(p, alpha=weight_decay)
                    p.add_(direction, alpha=-lr * lr_other_ratio)

        # Clear references
        self.in_acts.clear()
        self.out_grads.clear()
        self.sample_indices.clear()
        return loss

    def close(self):
        """
        Remove all hooks at the end of training
        """

        for h in self.hooks:
            h.remove()
        self.hooks.clear()

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import wandb
import time
import os
from typing import Dict, Any
import math

torch.backends.cudnn.benchmark = True

# ============================================================================
# Configuration
# ============================================================================

class Config:
    """Training configuration"""
    # Dataset
    dataset_name = 'CIFAR10'
    num_classes = 10
    data_dir = './data'
    
    # Model
    model_name = 'resnet18'
    
    num_epochs = 200
    batch_size = 128
    learning_rate = 0.1
    sample_size = 24
    lr_other_ratio = 1.0
    momentum = 0.9
    weight_decay = 5e-4

    use_sgd = False
    
    num_workers = 4
    
    lr_milestones = [60, 120, 160]
    lr_gamma = 0.2
    
    checkpoint_dir = './checkpoints'
    save_best_only = True
    
    wandb_project = 'cifar100-resnet18'
    wandb_name = None  # Will be auto-generated if None
    
    def __init__(self, **kwargs):
        """Allow overriding config parameters"""
        for key, value in kwargs.items():
            if hasattr(self, key):
                setattr(self, key, value)
            else:
                print(f"Warning: Unknown config parameter: {key}")

DEVICE = 'cuda'

# ============================================================================
# Model Definition - ResNet18
# ============================================================================

class BasicBlock(nn.Module):
    """Basic Block for ResNet18/34"""
    expansion = 1

    def __init__(self, in_channels, out_channels, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, 
                               stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3,
                               stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)
        
        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != self.expansion * out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, self.expansion * out_channels,
                         kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion * out_channels)
            )

    def forward(self, x):
        out = torch.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = torch.relu(out)
        return out


class ResNet(nn.Module):
    """ResNet architecture"""
    def __init__(self, block, num_blocks, num_classes=10):
        super(ResNet, self).__init__()
        self.in_channels = 64

        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        self.linear = nn.Linear(512 * block.expansion, num_classes)

    def _make_layer(self, block, out_channels, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_channels, out_channels, stride))
            self.in_channels = out_channels * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        out = torch.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = torch.nn.functional.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out


def ResNet18(num_classes=10):
    """ResNet18 model"""
    return ResNet(BasicBlock, [2, 2, 2, 2], num_classes=num_classes)


# ============================================================================
# Data Loading
# ============================================================================

def get_data_loaders(config: Config):
    """Create train and test data loaders"""
    
    # Data augmentation and normalization
    # CIFAR100 statistics
    mean = [0.4914, 0.4822, 0.4465]
    std = [0.2470, 0.2435, 0.2616]
    
    transform_train = transforms.Compose([
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean, std)
    ])
    
    transform_test = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean, std)
    ])
    
    trainset = torchvision.datasets.CIFAR10(
        root=config.data_dir,
        train=True,
        download=True,
        transform=transform_train
    )
    
    testset = torchvision.datasets.CIFAR10(
        root=config.data_dir,
        train=False,
        download=True,
        transform=transform_test
    )
    
    train_loader = DataLoader(
        trainset,
        batch_size=config.batch_size,
        shuffle=True,
        drop_last=True,
        num_workers=config.num_workers,
        persistent_workers=(config.num_workers > 0),
        pin_memory=True
    )
    
    test_loader = DataLoader(
        testset,
        batch_size=config.batch_size,
        shuffle=False,
        num_workers=config.num_workers,
        pin_memory=True
    )
    
    return train_loader, test_loader


# ============================================================================
# Training and Evaluation
# ============================================================================

def train_epoch(model, train_loader, criterion, optimizer, epoch):
    model.train()
    train_loss = 0
    correct = 0
    total = 0
    
    torch.cuda.synchronize()
    start_time = time.time()
    
    for batch_idx, (inputs, targets) in enumerate(train_loader):
        inputs, targets = inputs.to(DEVICE, non_blocking=True), targets.to(DEVICE, non_blocking=True)
        
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()
    
    avg_loss = train_loss / len(train_loader)
    accuracy = 100. * correct / total
    torch.cuda.synchronize()
    epoch_time = time.time() - start_time
    
    return avg_loss, accuracy, epoch_time


def evaluate(model, test_loader, criterion):
    """Evaluate on test set"""
    model.eval()
    test_loss = 0
    correct = 0
    total = 0
    
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(test_loader):
            inputs, targets = inputs.to(DEVICE, non_blocking=True), targets.to(DEVICE, non_blocking=True)
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            
            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
    
    avg_loss = test_loss / len(test_loader)
    accuracy = 100. * correct / total
    
    return avg_loss, accuracy


# ============================================================================
# Main Training Loop
# ============================================================================


def train_model(config: Config = None, **config_overrides):
    """
    Main training function
    
    Args:
        config: Config object, if None will create default
        **config_overrides: Override specific config parameters
    
    Example:
        # Use defaults
        train_model()
        
        # Override specific parameters
        train_model(num_epochs=100, batch_size=256)
        
        # Use custom config
        custom_config = Config(num_epochs=100, learning_rate=0.01)
        train_model(custom_config)
    """
    
    # Initialize config
    if config is None:
        config = Config(**config_overrides)
    else:
        # Apply overrides to existing config
        for key, value in config_overrides.items():
            if hasattr(config, key):
                setattr(config, key, value)
    
    # Create checkpoint directory
    os.makedirs(config.checkpoint_dir, exist_ok=True)
    
    # Initialize wandb
    wandb_name = config.wandb_name
    print(f"WANDB: {wandb_name}")
    wandb.init(
        project=config.wandb_project,
        name=wandb_name,
        config={
            'model': config.model_name,
            'dataset': config.dataset_name,
            'num_classes': config.num_classes,
            'num_epochs': config.num_epochs,
            'batch_size': config.batch_size,
            'learning_rate': config.learning_rate,
            'momentum': config.momentum,
            'weight_decay': config.weight_decay,
            'lr_milestones': config.lr_milestones,
            'lr_gamma': config.lr_gamma,
            'num_workers': config.num_workers,
        }
    )
    
    print("=" * 80)
    print(f"Training Configuration:")
    print(f"Dataset: {config.dataset_name}")
    print(f"Model: {config.model_name}")
    print(f"Batch size: {config.batch_size}")
    print(f"Learning rate: {config.learning_rate}")
    print(f"Epochs: {config.num_epochs}")
    print(f"LR Schedule: MultiStepLR(milestones={config.lr_milestones}, gamma={config.lr_gamma})")
    print("=" * 80)
    
    # Create data loaders
    print("\nLoading data...")
    train_loader, test_loader = get_data_loaders(config)
    print(f"Train samples: {len(train_loader.dataset)}")
    print(f"Test samples: {len(test_loader.dataset)}")
    
    # Create model
    print(f"\nBuilding {config.model_name}...")
    model = ResNet18(num_classes=config.num_classes)
    model = model.to(DEVICE)
    
    # Count parameters
    total_params = sum(p.numel() for p in model.parameters())
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    print(f"Total parameters: {total_params:,}")
    print(f"Trainable parameters: {trainable_params:,}")
    
    # Loss and optimizer
    criterion = nn.CrossEntropyLoss()
    if config.use_sgd:
        optimizer = optim.SGD(
            model.parameters(),
            lr=config.learning_rate,
            momentum=config.momentum,
            weight_decay=config.weight_decay
        )
    else:
        optimizer = Frog(
            model=model, 
            params=model.parameters(), 
            lr=config.learning_rate,
            lr_other_ratio=config.lr_other_ratio,
            weight_decay=config.weight_decay,
            denom_eps=1e-10,
            sample_size=config.sample_size,
            momentum=0.9,
            num_iters=4
        )
    
    # Learning rate scheduler
    scheduler = optim.lr_scheduler.MultiStepLR(
        optimizer,
        milestones=config.lr_milestones,
        gamma=config.lr_gamma
    )
    
    # Training loop
    best_acc = 0
    print("\nStarting training...\n")
    
    for epoch in range(1, config.num_epochs + 1):
        print(f"\n{'='*80}")
        print(f"Epoch {epoch}/{config.num_epochs}")
        print(f"Learning rate: {scheduler.get_last_lr()[0]:.6f}")
        print(f"{'='*80}")
        
        # Train
        train_loss, train_acc, epoch_time = train_epoch(
            model, train_loader, criterion, optimizer, epoch
        )
        
        # Evaluate
        test_loss, test_acc = evaluate(model, test_loader, criterion)
        
        # Step scheduler
        scheduler.step()
        
        # Print epoch summary
        print(f"\nEpoch {epoch} Summary:")
        print(f"Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.2f}%")
        print(f"Test Loss: {test_loss:.4f} | Test Acc: {test_acc:.2f}%")
        print(f"Time: {epoch_time:.2f}s")
        
        # Log to wandb
        wandb.log({
            'epoch': epoch,
            'train/loss': train_loss,
            'train/accuracy': train_acc,
            'test/loss': test_loss,
            'test/accuracy': test_acc,
            'learning_rate': scheduler.get_last_lr()[0],
            'epoch_time': epoch_time,
        }, commit=True)
        
        
        # Save checkpoint
        if test_acc > best_acc:
            print(f'Saving checkpoint... (Test Acc: {test_acc:.2f}% > Best: {best_acc:.2f}%)')
            state = {
                'model': model.state_dict(),
                'accuracy': test_acc,
                'epoch': epoch,
                'optimizer': optimizer.state_dict(),
                'scheduler': scheduler.state_dict(),
            }
            checkpoint_path = os.path.join(config.checkpoint_dir, 'best_model.pth')
            torch.save(state, checkpoint_path)
            best_acc = test_acc
            
            # Log best model to wandb
            wandb.run.summary["best_accuracy"] = best_acc
            wandb.run.summary["best_epoch"] = epoch
    
    # Save final model
    if not config.save_best_only:
        final_checkpoint_path = os.path.join(config.checkpoint_dir, 'final_model.pth')
        torch.save({
            'model': model.state_dict(),
            'accuracy': test_acc,
            'epoch': config.num_epochs,
        }, final_checkpoint_path)
    
    print("\n" + "="*80)
    print("Training completed!")
    print(f"Best test accuracy: {best_acc:.2f}%")
    print("="*80)
    
    wandb.finish()

    if not config.use_sgd:
        optimizer.close()
    
    return model, best_acc

In [ ]:
def get_config(name: str, use_sgd: bool, batch_size: int, learning_rate: float, weight_decay: float, sample_size: int = 24):
    return Config(
        num_epochs=160,
        use_sgd=use_sgd,
        learning_rate=learning_rate,
        lr_other_ratio=1.0,
        lr_milestones=[80, 120],
        lr_gamma=0.1,
        sample_size=sample_size,
        weight_decay=weight_decay,
        batch_size=batch_size,
        wandb_project='cifar10-newcomp',
        wandb_name=name
    )

configs = [
    get_config(name="SGD128", use_sgd=True, batch_size=128, learning_rate=0.1, weight_decay=1e-4),
    get_config(name="SGD512", use_sgd=True, batch_size=512, learning_rate=0.1, weight_decay=1e-4),
    get_config(name="FROG512s16", use_sgd=False, batch_size=512, learning_rate=0.01, weight_decay=1e-5, sample_size=16),
    get_config(name="FROG512s24", use_sgd=False, batch_size=512, learning_rate=0.01, weight_decay=1e-5, sample_size=24),
    get_config(name="FROG512s32", use_sgd=False, batch_size=512, learning_rate=0.01, weight_decay=1e-5, sample_size=32),
    get_config(name="FROG512s64", use_sgd=False, batch_size=512, learning_rate=0.01, weight_decay=1e-5, sample_size=64),
    get_config(name="FROG128s16", use_sgd=False, batch_size=128, learning_rate=0.01, weight_decay=1e-5, sample_size=16),
    get_config(name="FROG128s24", use_sgd=False, batch_size=128, learning_rate=0.01, weight_decay=1e-5, sample_size=24),
    get_config(name="FROG128s32", use_sgd=False, batch_size=128, learning_rate=0.01, weight_decay=1e-5, sample_size=32),
    get_config(name="FROG128s64", use_sgd=False, batch_size=128, learning_rate=0.01, weight_decay=1e-5, sample_size=64),
]

for config in configs:
    print(f"Training {config.wandb_name}")
    train_model(config)

wandb: [wandb.login()] Loaded credentials for https://api.wandb.ai from /home/coder/.netrc.


Training SGD512
WANDB: SGD512


wandb: Currently logged in as: nbreskanu73 to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Training Configuration:
Dataset: CIFAR10
Model: resnet18
Batch size: 512
Learning rate: 0.1
Epochs: 160
LR Schedule: MultiStepLR(milestones=[80, 120], gamma=0.1)

Loading data...
Train samples: 50000
Test samples: 10000

Building resnet18...
Total parameters: 11,173,962
Trainable parameters: 11,173,962

Starting training...


Epoch 1/160
Learning rate: 0.100000

Epoch 1 Summary:
Train Loss: 2.5540 | Train Acc: 18.81%
Test Loss: 1.8566 | Test Acc: 29.98%
Time: 6.60s
Saving checkpoint... (Test Acc: 29.98% > Best: 0.00%)

Epoch 2/160
Learning rate: 0.100000

Epoch 2 Summary:
Train Loss: 1.7361 | Train Acc: 35.08%
Test Loss: 1.5732 | Test Acc: 41.45%
Time: 5.59s
Saving checkpoint... (Test Acc: 41.45% > Best: 29.98%)

Epoch 3/160
Learning rate: 0.100000

Epoch 3 Summary:
Train Loss: 1.5185 | Train Acc: 43.73%
Test Loss: 1.4520 | Test Acc: 47.11%
Time: 5.01s
Saving checkpoint... (Test Acc: 47.11% > Best: 41.45%)

Epoch 4/160
Learning rate: 0.100000

Epoch 4 Summary:
Train Loss: 1.3528 | Trai

wandb: ERROR The nbformat package was not found. It is required to save notebook history.


...

Training FROG128s64
WANDB: FROG128s64


Training Configuration:
Dataset: CIFAR10
Model: resnet18
Batch size: 128
Learning rate: 0.01
Epochs: 160
LR Schedule: MultiStepLR(milestones=[80, 120], gamma=0.1)

Loading data...
Train samples: 50000
Test samples: 10000

Building resnet18...
Total parameters: 11,173,962
Trainable parameters: 11,173,962

Starting training...


Epoch 1/160
Learning rate: 0.010000

Epoch 1 Summary:
Train Loss: 1.2064 | Train Acc: 57.01%
Test Loss: 0.8327 | Test Acc: 72.61%
Time: 22.40s
Saving checkpoint... (Test Acc: 72.61% > Best: 0.00%)

Epoch 2/160
Learning rate: 0.010000

Epoch 2 Summary:
Train Loss: 0.6941 | Train Acc: 76.33%
Test Loss: 0.5847 | Test Acc: 80.50%
Time: 22.32s
Saving checkpoint... (Test Acc: 80.50% > Best: 72.61%)

Epoch 3/160
Learning rate: 0.010000

Epoch 3 Summary:
Train Loss: 0.5418 | Train Acc: 81.57%
Test Loss: 0.4956 | Test Acc: 83.81%
Time: 22.57s
Saving checkpoint... (Test Acc: 83.81% > Best: 80.50%)

Epoch 4/160
Learning rate: 0.010000

Epoch 4 Summary:
Train Loss: 0.4587 | 

wandb: ERROR The nbformat package was not found. It is required to save notebook history.



Epoch 160 Summary:
Train Loss: 0.0003 | Train Acc: 99.99%
Test Loss: 0.4242 | Test Acc: 94.12%
Time: 21.97s

Training completed!
Best test accuracy: 94.34%


epoch,▁▁▁▁▁▂▂▂▂▂▂▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇█████
epoch_time,▄▂▂▄▁▁▄▄▃▄▃▆▇▄▁▂▄▄▃▄▅▂▅▆▄▆▂▁▃▅▄▄▇▃▃▅█▇▄▅
learning_rate,██████████████████▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁
test/accuracy,▁▂▄▅▅▅▆▆▆▆▆▆▆▆▆▇▇▇▇▇▇▇▇███▇█████▇███████
test/loss,█▅▄▂▁▁▁▂▂▁▁▂▂▃▂▃▅▄▂▂▄▄▄▃▅▅▅▅▆▅▅▅▅▅▅▅▅▅▅▅
train/accuracy,▁▅▆▇▇███████████████████████████████████
train/loss,█▅▄▃▃▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
best_accuracy,94.34
best_epoch,136
epoch,160
epoch_time,21.97204
